In [ ]:
from medcat_den.injection import inject_into_medcat
from medcat_den.den import get_default_den
from medcat_den.utils import summarise_den
# NOTE: this will centralise model storage and
#       allow using the centralise storage automatically
inject_into_medcat(inject_save=True)
# to see available models
print(summarise_den(get_default_den()))
from medcat.cat import CAT
from medcat.stats import get_stats
import json


Den (LOCAL_USER) with 3 models:
a645742030cae7be ('No description\n[27 August 2025] BASE MODEL - FAKE')
 - History: ['a645742030cae7be']
 - Source ontologies: ['FAKE-ONT']
 - Meta models: []
 - MedCAT Version: 2.0.0
 - Last updated: 2025-08-27T14:19:48.479156
cbbc516986aa8e30 ('No description\n[27 August 2025] BASE MODEL - FAKE\n[27 August 2025] Additional-fake-data-trained')
 - History: ['a645742030cae7be', 'cbbc516986aa8e30']
 - Source ontologies: ['FAKE-ONT']
 - Meta models: []
 - MedCAT Version: 2.0.0
 - Last updated: 2025-08-27T14:25:22.993770
2d99c0efac4d5af7 ('N/A\n[02 September 2025] Brand New Base Model\n[02 September 2025] Empty Model w 2d99c0efac4d5af7')
 - History: ['2d99c0efac4d5af7']
 - Source ontologies: []
 - Meta models: []
 - MedCAT Version: 2.0.0
 - Last updated: 2025-09-02T12:39:23.477744


In [ ]:
# input data 

# make sure the export is in this folder
ann_dir = "../data/mct_exports/"
# CHANGE THIS:
mctrainer_export_name = "example_mct_export.json" 

mctrainer_export_path = ann_dir + mctrainer_export_name

# CHANGE THIS:
model_hash = 'a645742030cae7be'

# CHANGE THIS [optional]:
output_model_name = f"{model_hash}_trained_model"

# Add training filter if needed
snomed_filter_path = None  # path to snomed filter

In [5]:

# Create CAT - the main class from medcat used for concept annotation
cat = CAT.load_model_pack(model_hash)  # NOTE: loads from den automatically
cat.config.components.linking.filters.cuis = set()  # To remove exisitng filters


# Set filter

Setting a filter will speed up training time since it will only train the concepts you're interested in.

In [6]:
# Add extra training filter if required.
if snomed_filter_path:
    snomed_filter = set(json.load(open(snomed_filter_path)))
else:
    snomed_filter = set(cat.cdb.cui2info.keys())

# Train

In [10]:
with open(mctrainer_export_path) as f:
    data = json.load(f)
cat.trainer.train_supervised_raw(
    data=data, 
    nepochs=3,
    reset_cui_count=False,
    print_stats=True,
    use_filters=True,
    extra_cui_filter=snomed_filter,  # If not filter is set remove this line
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]




















Epoch:  33%|███▎      | 1/3 [00:03<00:06,  3.14s/it]


















Epoch:  67%|██████▋   | 2/3 [00:06<00:03,  3.02s/it]





















({}, {}, {}, {}, {}, {}, {}, {})

# Stats

In [13]:
fps, fns, tps, cui_prec, cui_rec, cui_f1, cui_counts, examples = get_stats(cat, data, use_project_filters=True)


Epoch: 0, Prec: 0.7795454545454545, Rec: 0.8511166253101737, F1: 0.8137603795966786

Docs with false positives: 1734; 611; 879; 485; 1605; 1377; 716; 1881; 496; 641

Docs with false negatives: 1734; 611; 879; 1605; 1377; 716; 1881; 496; 641; 898



False Positives

high~blood~pressure                                                    - C0020538             -         20
fever                                                                  - C03                  -         17
syndrome                                                               - C0039082             -         15
alcohol~abuse                                                          - C0085762             -         10
cardiovascular                                                         - C1301700             -          8
cardiac                                                                - C0018801             -          6
deficits                                                               - C0009241           

In [16]:
cui = "C0004096" # Asthma
print(cui_f1[cui], cui_prec[cui], cui_rec[cui])

1.0 1.0 1.0


# Seve model

In [ ]:
# you can save centrally (since tihs is injected) with
cat.save_model_pack(f"Fine-tuned with {mctrainer_export_name}")

/Users/martratas/Documents/CogStack/.MedCAT.nosync/cogstack-ops/medcat-scripts/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
